In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -U gensim
!pip install -U nltk
!pip install -U scikit-learn

import os

import csv

import nltk
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")
from nltk import *
from nltk.corpus import stopwords
from nltk import PorterStemmer
from nltk import cluster
from nltk.stem import WordNetLemmatizer
from nltk.cluster import euclidean_distance

from collections import Counter

import re

import numpy as np
from numpy import array

import pandas

import math

import sklearn
from sklearn.cluster import SpectralClustering, KMeans, AgglomerativeClustering
from sklearn.neighbors import kneighbors_graph

from gensim.models import Word2Vec


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# initialize average accuracy list
scores = []
for i in range(10):
  path = "/content/drive/MyDrive/product_reviews"

  sentences_all = []
  sentences_clean = []
  files = []

  # generate stopwords
  stops = Counter(stopwords.words())

  # initialize stemmer

  ps = PorterStemmer()
  for p in os.listdir(path):
    with open(path +"/" +p, "r") as f:
      if p != "README.txt":
        files.append(p)
        text = f.read()
        # preprocessing- remove all tags, non-alphanum chars, redundant spacing
        text = re.sub("##", "", text)
        text = re.sub("\[t\]", "", text)
        text = re.sub("\\n", "", text)
        text = re.sub("\[u\]", "", text)
        text = re.sub("\[(\+|\-)\d\]", " ", text)
        text = text.lower()
        sentences = text.split(".")
        sentences = [re.sub("[^A-Za-z0-9 ]+", "", sent) for sent in sentences]
        sentences = [re.sub("\s\s+", " ", sent) for sent in sentences]

        # remove stopwords, tokenize each sentence, stem each word in sentence
        for sent in sentences:
          sent = " ".join([word for word in sent.split() if word not in stops])
          tokens = word_tokenize(sent)
          sentences_clean.append([ps.stem(t) for t in tokens])

        sentences_all += sentences

  # flatten sentences, remove empty sentnences
  sentences_clean = [sentence for sentence in sentences_clean if sentence != []]

  # remove very short words
  flat_tokens = [word for sentence in sentences_clean for word in sentence if len(word) > 2]

  # get most common words
  fdist = FreqDist(flat_tokens)
  targets = [t[0] for t in fdist.most_common(50)]

  pseudos = []

  for target in fdist.most_common(50): 
    # for each common word, generate its reverse, append this to list of reversed words
    target_token = target[0]
    freq = target[1]
    pseudo = target_token[::-1]
    pseudos.append(pseudo)

    # find locations in corpus of all instances of target word
    target_locations = []
    for sentence_index, sentence in enumerate(sentences_clean):
      for token_index, token in enumerate(sentence):
        if token == target_token:
          target_locations.append((sentence_index, token_index))

    substitutions = []

    # replace half of those instances with the psuedo
    for i in range(int(freq/2)):
      substitution = numpy.random.randint(low=0, high=len(target_locations))
      substitutions.append(substitution)
      sentences_clean[target_locations[substitution][0]][target_locations[substitution][1]] = pseudo

  terms = targets + pseudos
  # generate word model based on clean vocabulary, increase learning rate
  word_model = Word2Vec(min_count=1,window=3,vector_size=72,sample=6e-5, alpha=0.15, min_alpha=0.0007)
  word_model.build_vocab(sentences_clean)
  word_model.train(sentences_clean, total_examples=word_model.corpus_count, epochs=32, report_delay=1)

  # make list of words of form [target0, psuedo0, target1, psuedo1, ... target49, psuedo49]
  vecs = []
  c_words = []
  for target in targets:
    c_words.append(target)
    vecs.append(word_model.wv[target])
    c_words.append(target[::-1])
    vecs.append(word_model.wv[target[::-1]])

  # apply kmeans clustering algorithm with 50 clusters, increase iteration sizes, get predictions after fitting
  clusterer = KMeans(n_clusters = 50, max_iter=600, n_init=20).fit_predict(vecs)

  # pair of words matches, then increase correct classification by one
  cor = 0
  for i in range(0,100,2):
    c1 = clusterer[i]
    c2 = clusterer[i+1]
    if c1 == c2:
      cor += 1
  avg = (cor/50)*100
  scores.append(avg)
  
print("average correct classification: " + str(numpy.mean(scores)) + "%")
print("standard deviation: " +str(numpy.std(scores)))

average correct classification: 78.0%
standard deviation: 3.5777087639996634
